In [47]:
path = '/home/akazemi3/Desktop/MB_Lab_Project/'
import sys
sys.path.append(path)


import torchvision
from models.layer_operations.output import Output
import torch
from torch import nn
torch.manual_seed(0)
torch.cuda.manual_seed(0)
model = torchvision.models.alexnet(pretrained=False)

In [61]:
class Model(nn.Module):
    
    
    def __init__(self,
                c5: nn.Module,
                r5: nn.Module,
                mp5: nn.Module,
                last: nn.Module,
                print_shape: bool = False
                ):
        
        super(Model, self).__init__()
        
        self.c5 = c5
        self.r5 = r5
        self.mp5 = mp5
        self.last = last
        self.print_shape = print_shape
        
        
    def forward(self, x:nn.Module):
                
        x = x.float()
        
        # extract activations from L4
        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook

        model.features[9].register_forward_hook(get_activation('features.9'))
        output = model(x)
        
        x = activation['features.9']
        if self.print_shape:
            print('layer 4', x.shape)        
        
        x = self.c5(x)
        if self.print_shape:
            print('conv5', x.shape)        
        
        x = self.r5(x)
        if self.print_shape:
            print('relu5', x.shape)        
        
        x = self.mp5(x)    
        if self.print_shape:
            print('maxpool5', x.shape)
        
        x = self.last(x)
        if self.print_shape:
            print('output', x.shape)
        
        return x    


    
class AlexnetUntrained:

    
    def __init__(self, filters_5 = 20000):
    
        self.filters_5 = filters_5        

    
    def Build(self):
        
        c5 = nn.Conv2d(256, self.filters_5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        r5 = nn.ReLU(inplace=True)
        mp5 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        last = Output()

        return Model(c5,r5,mp5,last)  

In [57]:
alexnetU = AlexnetUntrained(filters_5 = 50000).Build()

In [58]:
x = torch.rand(10,3,224,224)

In [60]:
alexnetU(x).shape

torch.Size([10, 50000, 6, 6])